In [1]:
import copy
import os.path
import sys

pwd = os.getcwd()
sys.path.append(os.path.dirname(pwd))

from utils.model_utils.evaluate import evaluate_model
from utils.model_utils.load_model import *
from utils.model_utils.model_config import ModelConfig
from utils.dataset_utils.load_dataset import load_data
from utils.decompose_utils.weight_remover import WeightRemoverBert
from utils.decompose_utils.concern_identification import ConcernIdentificationBert
from utils.decompose_utils.tangling_identification import TanglingIdentification
from transformers import AutoConfig
from utils.model_utils.save_module import save_module
from datetime import datetime
from utils.decompose_utils.concern_modularization import ConcernModularizationBert
from utils.decompose_utils.sampling import sampling_class
from utils.dataset_utils.load_dataset import convert_dataset_labels_to_binary, extract_and_convert_dataloader, TextDataset
from os.path import join as join
from torch.utils.data import DataLoader
from utils.model_utils.load_tokenizer import load_tokenizer
from utils.paths import p, get_dir
from datasets import load_dataset, DatasetDict
import torch
import torch

model_name = "sadickam/sdg-classification-bert"
model_type = "pretrained"
data = "OSDG"
num_labels = 16


# model_name = "textattack/bert-base-uncased-imdb"
# model_type = "pretrained"
# data = "IMDb"
# num_labels = 2

# model_name = "fabriceyhc/bert-base-uncased-yahoo_answers_topics"
# model_type = "pretrained"
# data = "Yahoo"
# num_labels = 10

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

checkpoint_name = None
config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
model_config = ModelConfig(
    _model_name=model_name,
    _model_type=model_type,
    _data=data,
    _transformer_config=config,
    _checkpoint_name=checkpoint_name,
    _device=device,
)

def pppp(module1, ci1, ti1, model_config):
    ConcernModularizationBert.channeling(module1, ci1.active_node, ti1.dead_node, i, model_config.device)
    from transformers import BertForSequenceClassification
    config1 = AutoConfig.from_pretrained(model_name)
    config1.id2label = {0: "negative", 1: "positive"}
    config1.label2id = {"negative": 0, "positive": 1}
    config1._num_labels=2
    module2 = BertForSequenceClassification(config1)
    module2 = module1.to(model_config.device)
    module2.load_state_dict(module1.state_dict())
    return module2

In [2]:
for i in range(num_labels):
    model, tokenizer, checkpoint = load_classification_model(model_config, train_mode=False)

    train_dataloader, valid_dataloader, test_dataloader = load_data(
        model_config, batch_size=32, test_size=0.3
    )
    print("Start Time:" + datetime.now().strftime("%H:%M:%S"))
    print("#Module " + str(i) + " in progress....")
    num_samples = 64
    
    positive_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )
    
    all_samples = sampling_class(
        train_dataloader, 200, 20, num_labels, False, 4, device=device
    )
    
    module1 = copy.deepcopy(model)
    w = WeightRemoverBert(model, p=0.9)
    ci1 = ConcernIdentificationBert(model, p=0.4)
    ti1 = TanglingIdentification(model, p=0.5)
    
    ff1 = [
        [torch.sum(model.bert.encoder.layer[num].intermediate.dense.weight != 0).item()]
        for num in range(config.num_hidden_layers)
    ]
    ff2 = [
        [torch.sum(model.bert.encoder.layer[num].output.dense.weight != 0).item()]
        for num in range(config.num_hidden_layers)
    ]
    pooler = [torch.sum(model.bert.pooler.dense.weight != 0).item()]
    classifier = [torch.sum(model.classifier.weight != 0).item()]
    print("origin")
    # result = evaluate_model(model, model_config, test_dataloader)
    
    print("Start Positive CI sparse")
    
    for batch in all_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = w.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())
        classifier.append(torch.sum(module1.classifier.weight != 0).item())
    
        # result = evaluate_model(module1, model_config, test_dataloader)
    
    print("Start Positive CI after sparse")
    
    for batch in positive_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = ci1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())
        classifier.append(torch.sum(module1.classifier.weight != 0).item())
    
        # result = evaluate_model(module1, model_config, test_dataloader)
    
    print("Start Negative TI")
    
    for batch in negative_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t = ti1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())
        classifier.append(torch.sum(module1.classifier.weight != 0).item())
        # result = evaluate_model(module1, model_config, test_dataloader)

    collected_input_ids = []
    collected_attention_mask = []
    collected_labels = []
    count = 0
    
    for batch in test_dataloader:
        if count >= 100:
            break
    
        input_ids = batch["input_ids"].to(model_config.device)
        attention_mask = batch["attention_mask"].to(model_config.device)
        labels = batch["labels"].to(model_config.device)
    
        # Add data to lists
        collected_input_ids.append(input_ids)
        collected_attention_mask.append(attention_mask)
        collected_labels.append(labels)
    
        # Increment the count by the batch size
        count += input_ids.size(0)
    
    converted_train_dataloader = extract_and_convert_dataloader(train_dataloader, i, m)
    converted_valid_dataloader = extract_and_convert_dataloader(valid_dataloader, i, m)
    converted_test_dataloader = extract_and_convert_dataloader(test_dataloader, i, m)
    
    module2 = pppp(module1, ci1, ti1, model_config)
    print(i, m)
    result = evaluate_model(module2, model_config, converted_test_dataloader)


Directory /home/Minwoo/LESN/Decompose/DecomposeBERT/Models/Configs/pretrained/sadickam/sdg-classification-bert exists.
Loading the model.
Start Time:17:16:21
#Module 0 in progress....
origin
Start Positive CI sparse
Start Positive CI after sparse
Start Negative TI
[1, 2, 7, 9, 10, 12, 15]
[0]
[0, 9, 10, 0, 0, 0, 0, 11, 0, 16, 16, 0, 9, 0, 0, 16]
[16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0 1


Evaluating: 100%|██████████| 50/50 [00:16<00:00,  3.01it/s]


Loss: 0.5743
Precision: 0.7662, Recall: 0.7047, F1-Score: 0.6890
              precision    recall  f1-score   support

           0       0.89      0.47      0.61       775
           1       0.65      0.94      0.77       797

    accuracy                           0.71      1572
   macro avg       0.77      0.70      0.69      1572
weighted avg       0.76      0.71      0.69      1572

Directory /home/Minwoo/LESN/Decompose/DecomposeBERT/Models/Configs/pretrained/sadickam/sdg-classification-bert exists.
Loading the model.
Start Time:17:17:12
#Module 0 in progress....
origin
Start Positive CI sparse
Start Positive CI after sparse
Start Negative TI
[2, 7, 9, 10, 15]
[0]
[0, 0, 11, 0, 0, 0, 0, 14, 0, 16, 14, 0, 0, 0, 0, 16]
[16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0 2


Evaluating: 100%|██████████| 50/50 [00:16<00:00,  3.11it/s]


Loss: 0.2470
Precision: 0.9035, Recall: 0.9033, F1-Score: 0.9033
              precision    recall  f1-score   support

           0       0.89      0.92      0.90       795
           1       0.91      0.89      0.90       797

    accuracy                           0.90      1592
   macro avg       0.90      0.90      0.90      1592
weighted avg       0.90      0.90      0.90      1592

Directory /home/Minwoo/LESN/Decompose/DecomposeBERT/Models/Configs/pretrained/sadickam/sdg-classification-bert exists.
Loading the model.
Start Time:17:17:58
#Module 0 in progress....
origin
Start Positive CI sparse
Start Positive CI after sparse
Start Negative TI
[1, 2, 7, 9, 10, 15]
[0]
[0, 10, 13, 0, 0, 0, 0, 13, 0, 16, 16, 0, 0, 0, 0, 16]
[16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0 3


...

Evaluating: 100%|██████████| 45/45 [00:35<00:00,  1.28it/s]


Loss: 0.1876
Precision: 0.9466, Recall: 0.9537, F1-Score: 0.9501
              precision    recall  f1-score   support

           0       0.98      0.97      0.98      1110
           1       0.91      0.93      0.92       312

    accuracy                           0.97      1422
   macro avg       0.95      0.95      0.95      1422
weighted avg       0.97      0.97      0.97      1422

Directory /home/Minwoo/LESN/Decompose/DecomposeBERT/Models/Configs/pretrained/sadickam/sdg-classification-bert exists.
Loading the model.
Start Time:20:20:05
#Module 11 in progress....
origin
Start Positive CI sparse
Start Positive CI after sparse
Start Negative TI
[0, 1, 2, 7, 8, 9, 10, 12, 15]
[11, 6]
[15, 8, 8, 0, 0, 0, 0, 14, 12, 16, 15, 0, 8, 0, 0, 16]
[0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 16, 0, 0, 0, 0]
11 4


Evaluating: 100%|██████████| 50/50 [00:41<00:00,  1.19it/s]


Loss: 0.2301
Precision: 0.9250, Recall: 0.9557, F1-Score: 0.9393
              precision    recall  f1-score   support

           0       0.99      0.96      0.97      1260
           1       0.86      0.95      0.90       312

    accuracy                           0.96      1572
   macro avg       0.92      0.96      0.94      1572
weighted avg       0.96      0.96      0.96      1572

Directory /home/Minwoo/LESN/Decompose/DecomposeBERT/Models/Configs/pretrained/sadickam/sdg-classification-bert exists.
Loading the model.
Start Time:20:21:41
#Module 11 in progress....
origin
Start Positive CI sparse
Start Positive CI after sparse
Start Negative TI


KeyboardInterrupt: 